### Run in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1A4AzWlRKqWStMBpXR_DW7EBuL_pOU035)

### Setup your API key

Before you can use the OpenAI API, you must first obtain an API key. If you don't already have one, create a key for OPENAI_API_KEY which is obtained from: https://openai.com/blog/openai-api

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `OPENAI_API_KEY`.


### Mount this Colab notebook at Google Drive
To ensure your work is saved and can be accessed later, it's a good practice to mount your Google Drive to this Colab notebook. You can do this by running the following code. This will prompt you to authorize Colab to access your Google Drive account. After authorization, your Google Drive will be mounted, and you can save your files directly to it.

In [ ]:
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Mount google drive in the virtual envrionment
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ENVIRONMENT SETUP

In [ ]:
!pip install autogen-agentchat~=0.2
!pip install openai
!pip install httpx==0.27.2
!pip install dask[dataframe]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.2/382.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
from autogen import ConversableAgent, AssistantAgent

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Image

import autogen
import openai
client = openai.OpenAI(api_key=api_key)

### Download Essential Data

If this is the first time you run the code, make sure you set download_data to True

In [ ]:
import os

# Specify the output folder where the CSV will be saved
output_folder = "/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/"

# Check if the output folder exists and create it if it does not
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# Google Drive download commands using gdown for each file
!gdown --id 1-szLXRrvPaCguAwpJynA3dDUh83_BU5o -O "/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/RefBldgSmallOfficePost1980_v1.4_7.2_2B_USA_AZ_PHOENIX.csv"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-szLXRrvPaCguAwpJynA3dDUh83_BU5o
To: /content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/RefBldgSmallOfficePost1980_v1.4_7.2_2B_USA_AZ_PHOENIX_1.csv
100% 2.82M/2.82M [00:00<00:00, 143MB/s]


### Agent Definition

In [ ]:
llm_config = {"config_list": [{"model": "gpt-4o", "api_key": api_key}], "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/',
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
)

coder = autogen.AssistantAgent(
    name="Coder",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. You are a helpful assistant highly skilled in evaluating the quality of a given visualization code by providing a score from 1 (bad) - 10 (good) while providing clear rationale. YOU MUST CONSIDER VISUALIZATION BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
- bugs (bugs):  are there bugs, logic errors, syntax error or typos? Are there any reasons why the code may fail to compile? How should it be fixed? If ANY bug exists, the bug score MUST be less than 5.
- Data transformation (transformation): Is the data transformed appropriately for the visualization type? E.g., is the dataset appropriated filtered, aggregated, or grouped  if needed? If a date field is used, is the date field first converted to a date object etc?
- Goal compliance (compliance): how well the code meets the specified visualization goals?
- Visualization type (type): CONSIDERING BEST PRACTICES, is the visualization type appropriate for the data and intent? Is there a visualization type that would be more effective in conveying insights? If a different visualization type is more appropriate, the score MUST BE LESS THAN 5.
- Data encoding (encoding): Is the data encoded appropriately for the visualization type?
- aesthetics (aesthetics): Are the aesthetics of the visualization appropriate for the visualization type and the data?

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
Do not suggest code.
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code. Return "TERMINATE" if the figure meets the critique and is successfully generated.
""",
    llm_config=llm_config,
)

building_expert = autogen.AssistantAgent(
    name="Building_Expert",
    llm_config=llm_config,
    system_message="""You are an expert with domain knowledge in building energy. You don't write code, but evaluate the physcal meaning and domain knowledge of telling whether the generated code and visualization make sense.""",
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, critic, building_expert], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="get data locally from '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/RefBldgSmallOfficePost1980_v1.4_7.2_2B_USA_AZ_PHOENIX.csv' and plot a visualization of pie chart for enduses of the building. Save the generated figure to '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/'",
)

User_proxy (to chat_manager):

get data locally from '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/RefBldgSmallOfficePost1980_v1.4_7.2_2B_USA_AZ_PHOENIX.csv' and plot a visualization of pie chart for enduses of the building. Save the generated figure to '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/'

--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

To create a pie chart visualization of the building end uses from the specified CSV file and save it to the desired location, follow these steps:

1. Load the CSV file from the specified path.
2. Extract the relevant data for the end uses from the CSV file.
3. Plot the data using a pie chart.
4. Save the generated pie chart to the specified directory.

Let's execute the code that performs these tasks:

```python
import pandas as pd
import matplotlib.pyplot as plt

# Load CSV data into a DataFrame
file_path = '/content/drive/MyDrive/LL

ChatResult(chat_id=None, chat_history=[{'content': "get data locally from '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/RefBldgSmallOfficePost1980_v1.4_7.2_2B_USA_AZ_PHOENIX.csv' and plot a visualization of pie chart for enduses of the building. Save the generated figure to '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/'", 'role': 'assistant', 'name': 'User_proxy'}, {'content': "To create a pie chart visualization of the building end uses from the specified CSV file and save it to the desired location, follow these steps:\n\n1. Load the CSV file from the specified path.\n2. Extract the relevant data for the end uses from the CSV file.\n3. Plot the data using a pie chart.\n4. Save the generated pie chart to the specified directory.\n\nLet's execute the code that performs these tasks:\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load CSV data into a DataFrame\nfile_path = '/content/drive/MyDrive/LLM_Data_Analysis_and_Visualization/Re